 install Unsloth

In [ ]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

use mistral 7b instruct 4bits quant

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
    "unsloth/gemma-2b-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "teknium/OpenHermes-2.5-Mistral-7B", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

Unsloth: You passed in `teknium/OpenHermes-2.5-Mistral-7B` and `load_in_4bit = True`.
We shall load `unsloth/OpenHermes-2.5-Mistral-7B-bnb-4bit` for 4x faster loading.


config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.4
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Need 4.6gb vram

INPUT PREPARATION

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task of layout question answering , paired with an input that provides further context that contain  a list of extracted blocks from that image then followed by a list of theirs bounding boxes represented by 4 point's cordinates of them in that image ,. Write a response that appropriately completes the request .

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass



Inference

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "what is her name",
        "['REPUBLIOUE FRANCAISE',  'FR',  'CARTENATIONALEDIDENTITE/IDENTITYCARD',  'NOM/Sumame',  'MARTIN',  'Prenoms/Given.names',  'Maelys-Gaelle,Marie',  'SEXE/Sex',  'NATIONALITE/Nationalit',  'DATE DE NAlSS.YDate of birth',  'FRA',  '13071990',  'LIEUDE NAISSANCE/Place of birth',  'PARIS',  'NOM D'USAGE/ Altenate name',  'NOM D USAGE',  'NDU DOCUMENTLDocument No',  'DATE DEXPIR./Expiry date',  'X4RTBPFW4',  '11022030',  'Gnature',  '384213'],[[[184.0, 44.0], [608.0, 46.0], [608.0, 70.0], [184.0, 68.0]],[[668.0, 53.0], [717.0, 53.0], [717.0, 80.0], [668.0, 80.0]],[[185.0, 80.0], [609.0, 80.0], [609.0, 93.0], [185.0, 93.0]],[[313.0, 102.0], [408.0, 105.0], [408.0, 120.0], [313.0, 118.0]],[[317.0, 120.0], [431.0, 120.0], [431.0, 144.0], [317.0, 144.0]],[[319.0, 173.0], [447.0, 173.0], [447.0, 187.0], [319.0, 187.0]],[[317.0, 187.0], [558.0, 186.0], [558.0, 206.0], [317.0, 207.0]],[[317.0, 234.0], [388.0, 234.0], [388.0, 251.0], [317.0, 251.0]],[[410.0, 236.0], [553.0, 236.0], [553.0, 250.0], [410.0, 250.0]],[[585.0, 235.0], [758.0, 235.0], [758.0, 249.0], [585.0, 249.0]],[[408.0, 252.0], [456.0, 252.0], [456.0, 270.0], [408.0, 270.0]],[[585.0, 250.0], [711.0, 247.0], [711.0, 268.0], [585.0, 271.0]],[[319.0, 276.0], [521.0, 276.0], [521.0, 290.0], [319.0, 290.0]],[[317.0, 291.0], [388.0, 291.0], [388.0, 312.0], [317.0, 312.0]],[[318.0, 315.0], [494.0, 316.0], [494.0, 330.0], [318.0, 329.0]],[[319.0, 331.0], [493.0, 331.0], [493.0, 351.0], [319.0, 351.0]],[[320.0, 359.0], [511.0, 359.0], [511.0, 373.0], [320.0, 373.0]],[[585.0, 357.0], [744.0, 357.0], [744.0, 371.0], [585.0, 371.0]],[[319.0, 377.0], [531.0, 377.0], [531.0, 401.0], [319.0, 401.0]],[[585.0, 371.0], [711.0, 371.0], [711.0, 392.0], [585.0, 392.0]],[[390.0, 422.0], [496.0, 422.0], [496.0, 446.0], [390.0, 446.0]],[[588.0, 422.0], [725.0, 425.0], [725.0, 452.0], [588.0, 450.0]]]", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)[0].split("Response")[-1]

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


':\n\nThe name of the person in the image is Maelys-Gaelle Marie Martin.<|im_end|>'

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "where she is borned",
        "['REPUBLIOUE FRANCAISE',  'FR',  'CARTENATIONALEDIDENTITE/IDENTITYCARD',  'NOM/Sumame',  'MARTIN',  'Prenoms/Given.names',  'Maelys-Gaelle,Marie',  'SEXE/Sex',  'NATIONALITE/Nationalit',  'DATE DE NAlSS.YDate of birth',  'FRA',  '13071990',  'LIEUDE NAISSANCE/Place of birth',  'PARIS',  'NOM D'USAGE/ Altenate name',  'NOM D USAGE',  'NDU DOCUMENTLDocument No',  'DATE DEXPIR./Expiry date',  'X4RTBPFW4',  '11022030',  'Gnature',  '384213'],[[[184.0, 44.0], [608.0, 46.0], [608.0, 70.0], [184.0, 68.0]],[[668.0, 53.0], [717.0, 53.0], [717.0, 80.0], [668.0, 80.0]],[[185.0, 80.0], [609.0, 80.0], [609.0, 93.0], [185.0, 93.0]],[[313.0, 102.0], [408.0, 105.0], [408.0, 120.0], [313.0, 118.0]],[[317.0, 120.0], [431.0, 120.0], [431.0, 144.0], [317.0, 144.0]],[[319.0, 173.0], [447.0, 173.0], [447.0, 187.0], [319.0, 187.0]],[[317.0, 187.0], [558.0, 186.0], [558.0, 206.0], [317.0, 207.0]],[[317.0, 234.0], [388.0, 234.0], [388.0, 251.0], [317.0, 251.0]],[[410.0, 236.0], [553.0, 236.0], [553.0, 250.0], [410.0, 250.0]],[[585.0, 235.0], [758.0, 235.0], [758.0, 249.0], [585.0, 249.0]],[[408.0, 252.0], [456.0, 252.0], [456.0, 270.0], [408.0, 270.0]],[[585.0, 250.0], [711.0, 247.0], [711.0, 268.0], [585.0, 271.0]],[[319.0, 276.0], [521.0, 276.0], [521.0, 290.0], [319.0, 290.0]],[[317.0, 291.0], [388.0, 291.0], [388.0, 312.0], [317.0, 312.0]],[[318.0, 315.0], [494.0, 316.0], [494.0, 330.0], [318.0, 329.0]],[[319.0, 331.0], [493.0, 331.0], [493.0, 351.0], [319.0, 351.0]],[[320.0, 359.0], [511.0, 359.0], [511.0, 373.0], [320.0, 373.0]],[[585.0, 357.0], [744.0, 357.0], [744.0, 371.0], [585.0, 371.0]],[[319.0, 377.0], [531.0, 377.0], [531.0, 401.0], [319.0, 401.0]],[[585.0, 371.0], [711.0, 371.0], [711.0, 392.0], [585.0, 392.0]],[[390.0, 422.0], [496.0, 422.0], [496.0, 446.0], [390.0, 446.0]],[[588.0, 422.0], [725.0, 425.0], [725.0, 452.0], [588.0, 450.0]]]", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)[0].split("Response")[-1]

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


':\n\nShe was born in Paris.<|im_end|>'

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "where she is borned(country)",
        "['REPUBLIOUE FRANCAISE',  'FR',  'CARTENATIONALEDIDENTITE/IDENTITYCARD',  'NOM/Sumame',  'MARTIN',  'Prenoms/Given.names',  'Maelys-Gaelle,Marie',  'SEXE/Sex',  'NATIONALITE/Nationalit',  'DATE DE NAlSS.YDate of birth',  'FRA',  '13071990',  'LIEUDE NAISSANCE/Place of birth',  'PARIS',  'NOM D'USAGE/ Altenate name',  'NOM D USAGE',  'NDU DOCUMENTLDocument No',  'DATE DEXPIR./Expiry date',  'X4RTBPFW4',  '11022030',  'Gnature',  '384213'],[[[184.0, 44.0], [608.0, 46.0], [608.0, 70.0], [184.0, 68.0]],[[668.0, 53.0], [717.0, 53.0], [717.0, 80.0], [668.0, 80.0]],[[185.0, 80.0], [609.0, 80.0], [609.0, 93.0], [185.0, 93.0]],[[313.0, 102.0], [408.0, 105.0], [408.0, 120.0], [313.0, 118.0]],[[317.0, 120.0], [431.0, 120.0], [431.0, 144.0], [317.0, 144.0]],[[319.0, 173.0], [447.0, 173.0], [447.0, 187.0], [319.0, 187.0]],[[317.0, 187.0], [558.0, 186.0], [558.0, 206.0], [317.0, 207.0]],[[317.0, 234.0], [388.0, 234.0], [388.0, 251.0], [317.0, 251.0]],[[410.0, 236.0], [553.0, 236.0], [553.0, 250.0], [410.0, 250.0]],[[585.0, 235.0], [758.0, 235.0], [758.0, 249.0], [585.0, 249.0]],[[408.0, 252.0], [456.0, 252.0], [456.0, 270.0], [408.0, 270.0]],[[585.0, 250.0], [711.0, 247.0], [711.0, 268.0], [585.0, 271.0]],[[319.0, 276.0], [521.0, 276.0], [521.0, 290.0], [319.0, 290.0]],[[317.0, 291.0], [388.0, 291.0], [388.0, 312.0], [317.0, 312.0]],[[318.0, 315.0], [494.0, 316.0], [494.0, 330.0], [318.0, 329.0]],[[319.0, 331.0], [493.0, 331.0], [493.0, 351.0], [319.0, 351.0]],[[320.0, 359.0], [511.0, 359.0], [511.0, 373.0], [320.0, 373.0]],[[585.0, 357.0], [744.0, 357.0], [744.0, 371.0], [585.0, 371.0]],[[319.0, 377.0], [531.0, 377.0], [531.0, 401.0], [319.0, 401.0]],[[585.0, 371.0], [711.0, 371.0], [711.0, 392.0], [585.0, 392.0]],[[390.0, 422.0], [496.0, 422.0], [496.0, 446.0], [390.0, 446.0]],[[588.0, 422.0], [725.0, 425.0], [725.0, 452.0], [588.0, 450.0]]]", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)[0].split("Response")[-1]

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


':\n\nThe person in the image was born in Paris, France.<|im_end|>'

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "where she is borned(continent)",
        "['REPUBLIOUE FRANCAISE',  'FR',  'CARTENATIONALEDIDENTITE/IDENTITYCARD',  'NOM/Sumame',  'MARTIN',  'Prenoms/Given.names',  'Maelys-Gaelle,Marie',  'SEXE/Sex',  'NATIONALITE/Nationalit',  'DATE DE NAlSS.YDate of birth',  'FRA',  '13071990',  'LIEUDE NAISSANCE/Place of birth',  'PARIS',  'NOM D'USAGE/ Altenate name',  'NOM D USAGE',  'NDU DOCUMENTLDocument No',  'DATE DEXPIR./Expiry date',  'X4RTBPFW4',  '11022030',  'Gnature',  '384213'],[[[184.0, 44.0], [608.0, 46.0], [608.0, 70.0], [184.0, 68.0]],[[668.0, 53.0], [717.0, 53.0], [717.0, 80.0], [668.0, 80.0]],[[185.0, 80.0], [609.0, 80.0], [609.0, 93.0], [185.0, 93.0]],[[313.0, 102.0], [408.0, 105.0], [408.0, 120.0], [313.0, 118.0]],[[317.0, 120.0], [431.0, 120.0], [431.0, 144.0], [317.0, 144.0]],[[319.0, 173.0], [447.0, 173.0], [447.0, 187.0], [319.0, 187.0]],[[317.0, 187.0], [558.0, 186.0], [558.0, 206.0], [317.0, 207.0]],[[317.0, 234.0], [388.0, 234.0], [388.0, 251.0], [317.0, 251.0]],[[410.0, 236.0], [553.0, 236.0], [553.0, 250.0], [410.0, 250.0]],[[585.0, 235.0], [758.0, 235.0], [758.0, 249.0], [585.0, 249.0]],[[408.0, 252.0], [456.0, 252.0], [456.0, 270.0], [408.0, 270.0]],[[585.0, 250.0], [711.0, 247.0], [711.0, 268.0], [585.0, 271.0]],[[319.0, 276.0], [521.0, 276.0], [521.0, 290.0], [319.0, 290.0]],[[317.0, 291.0], [388.0, 291.0], [388.0, 312.0], [317.0, 312.0]],[[318.0, 315.0], [494.0, 316.0], [494.0, 330.0], [318.0, 329.0]],[[319.0, 331.0], [493.0, 331.0], [493.0, 351.0], [319.0, 351.0]],[[320.0, 359.0], [511.0, 359.0], [511.0, 373.0], [320.0, 373.0]],[[585.0, 357.0], [744.0, 357.0], [744.0, 371.0], [585.0, 371.0]],[[319.0, 377.0], [531.0, 377.0], [531.0, 401.0], [319.0, 401.0]],[[585.0, 371.0], [711.0, 371.0], [711.0, 392.0], [585.0, 392.0]],[[390.0, 422.0], [496.0, 422.0], [496.0, 446.0], [390.0, 446.0]],[[588.0, 422.0], [725.0, 425.0], [725.0, 452.0], [588.0, 450.0]]]", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)[0].split("Response")[-1]

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


':\n\nThe person was born in Paris, which is located in the continent of Europe.<|im_end|>'

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "what is the document number",
        "['REPUBLIOUE FRANCAISE',  'FR',  'CARTENATIONALEDIDENTITE/IDENTITYCARD',  'NOM/Sumame',  'MARTIN',  'Prenoms/Given.names',  'Maelys-Gaelle,Marie',  'SEXE/Sex',  'NATIONALITE/Nationalit',  'DATE DE NAlSS.YDate of birth',  'FRA',  '13071990',  'LIEUDE NAISSANCE/Place of birth',  'PARIS',  'NOM D'USAGE/ Altenate name',  'NOM D USAGE',  'NDU DOCUMENTLDocument No',  'DATE DEXPIR./Expiry date',  'X4RTBPFW4',  '11022030',  'Gnature',  '384213'],[[[184.0, 44.0], [608.0, 46.0], [608.0, 70.0], [184.0, 68.0]],[[668.0, 53.0], [717.0, 53.0], [717.0, 80.0], [668.0, 80.0]],[[185.0, 80.0], [609.0, 80.0], [609.0, 93.0], [185.0, 93.0]],[[313.0, 102.0], [408.0, 105.0], [408.0, 120.0], [313.0, 118.0]],[[317.0, 120.0], [431.0, 120.0], [431.0, 144.0], [317.0, 144.0]],[[319.0, 173.0], [447.0, 173.0], [447.0, 187.0], [319.0, 187.0]],[[317.0, 187.0], [558.0, 186.0], [558.0, 206.0], [317.0, 207.0]],[[317.0, 234.0], [388.0, 234.0], [388.0, 251.0], [317.0, 251.0]],[[410.0, 236.0], [553.0, 236.0], [553.0, 250.0], [410.0, 250.0]],[[585.0, 235.0], [758.0, 235.0], [758.0, 249.0], [585.0, 249.0]],[[408.0, 252.0], [456.0, 252.0], [456.0, 270.0], [408.0, 270.0]],[[585.0, 250.0], [711.0, 247.0], [711.0, 268.0], [585.0, 271.0]],[[319.0, 276.0], [521.0, 276.0], [521.0, 290.0], [319.0, 290.0]],[[317.0, 291.0], [388.0, 291.0], [388.0, 312.0], [317.0, 312.0]],[[318.0, 315.0], [494.0, 316.0], [494.0, 330.0], [318.0, 329.0]],[[319.0, 331.0], [493.0, 331.0], [493.0, 351.0], [319.0, 351.0]],[[320.0, 359.0], [511.0, 359.0], [511.0, 373.0], [320.0, 373.0]],[[585.0, 357.0], [744.0, 357.0], [744.0, 371.0], [585.0, 371.0]],[[319.0, 377.0], [531.0, 377.0], [531.0, 401.0], [319.0, 401.0]],[[585.0, 371.0], [711.0, 371.0], [711.0, 392.0], [585.0, 392.0]],[[390.0, 422.0], [496.0, 422.0], [496.0, 446.0], [390.0, 446.0]],[[588.0, 422.0], [725.0, 425.0], [725.0, 452.0], [588.0, 450.0]]]", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)[0].split("Response")[-1]

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


':\n\nThe document number is X4RTBPFW4.<|im_end|>'

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "what is the title ",
        "['REPUBLIOUE FRANCAISE',  'FR',  'CARTENATIONALEDIDENTITE/IDENTITYCARD',  'NOM/Sumame',  'MARTIN',  'Prenoms/Given.names',  'Maelys-Gaelle,Marie',  'SEXE/Sex',  'NATIONALITE/Nationalit',  'DATE DE NAlSS.YDate of birth',  'FRA',  '13071990',  'LIEUDE NAISSANCE/Place of birth',  'PARIS',  'NOM D'USAGE/ Altenate name',  'NOM D USAGE',  'NDU DOCUMENTLDocument No',  'DATE DEXPIR./Expiry date',  'X4RTBPFW4',  '11022030',  'Gnature',  '384213'],[[[184.0, 44.0], [608.0, 46.0], [608.0, 70.0], [184.0, 68.0]],[[668.0, 53.0], [717.0, 53.0], [717.0, 80.0], [668.0, 80.0]],[[185.0, 80.0], [609.0, 80.0], [609.0, 93.0], [185.0, 93.0]],[[313.0, 102.0], [408.0, 105.0], [408.0, 120.0], [313.0, 118.0]],[[317.0, 120.0], [431.0, 120.0], [431.0, 144.0], [317.0, 144.0]],[[319.0, 173.0], [447.0, 173.0], [447.0, 187.0], [319.0, 187.0]],[[317.0, 187.0], [558.0, 186.0], [558.0, 206.0], [317.0, 207.0]],[[317.0, 234.0], [388.0, 234.0], [388.0, 251.0], [317.0, 251.0]],[[410.0, 236.0], [553.0, 236.0], [553.0, 250.0], [410.0, 250.0]],[[585.0, 235.0], [758.0, 235.0], [758.0, 249.0], [585.0, 249.0]],[[408.0, 252.0], [456.0, 252.0], [456.0, 270.0], [408.0, 270.0]],[[585.0, 250.0], [711.0, 247.0], [711.0, 268.0], [585.0, 271.0]],[[319.0, 276.0], [521.0, 276.0], [521.0, 290.0], [319.0, 290.0]],[[317.0, 291.0], [388.0, 291.0], [388.0, 312.0], [317.0, 312.0]],[[318.0, 315.0], [494.0, 316.0], [494.0, 330.0], [318.0, 329.0]],[[319.0, 331.0], [493.0, 331.0], [493.0, 351.0], [319.0, 351.0]],[[320.0, 359.0], [511.0, 359.0], [511.0, 373.0], [320.0, 373.0]],[[585.0, 357.0], [744.0, 357.0], [744.0, 371.0], [585.0, 371.0]],[[319.0, 377.0], [531.0, 377.0], [531.0, 401.0], [319.0, 401.0]],[[585.0, 371.0], [711.0, 371.0], [711.0, 392.0], [585.0, 392.0]],[[390.0, 422.0], [496.0, 422.0], [496.0, 446.0], [390.0, 446.0]],[[588.0, 422.0], [725.0, 425.0], [725.0, 452.0], [588.0, 450.0]]]", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)[0].split("Response")[-1]

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


':\n\nThe title of the image is "REPUBLIOUE FRANCAISE".<|im_end|>'

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "what is the gender",
        "['REPUBLIOUE FRANCAISE',  'FR',  'CARTENATIONALEDIDENTITE/IDENTITYCARD',  'NOM/Sumame',  'MARTIN',  'Prenoms/Given.names',  'Maelys-Gaelle,Marie',  'SEXE/Sex',  'NATIONALITE/Nationalit',  'DATE DE NAlSS.YDate of birth',  'FRA',  '13071990',  'LIEUDE NAISSANCE/Place of birth',  'PARIS',  'NOM D'USAGE/ Altenate name',  'NOM D USAGE',  'NDU DOCUMENTLDocument No',  'DATE DEXPIR./Expiry date',  'X4RTBPFW4',  '11022030',  'Gnature',  '384213'],[[[184.0, 44.0], [608.0, 46.0], [608.0, 70.0], [184.0, 68.0]],[[668.0, 53.0], [717.0, 53.0], [717.0, 80.0], [668.0, 80.0]],[[185.0, 80.0], [609.0, 80.0], [609.0, 93.0], [185.0, 93.0]],[[313.0, 102.0], [408.0, 105.0], [408.0, 120.0], [313.0, 118.0]],[[317.0, 120.0], [431.0, 120.0], [431.0, 144.0], [317.0, 144.0]],[[319.0, 173.0], [447.0, 173.0], [447.0, 187.0], [319.0, 187.0]],[[317.0, 187.0], [558.0, 186.0], [558.0, 206.0], [317.0, 207.0]],[[317.0, 234.0], [388.0, 234.0], [388.0, 251.0], [317.0, 251.0]],[[410.0, 236.0], [553.0, 236.0], [553.0, 250.0], [410.0, 250.0]],[[585.0, 235.0], [758.0, 235.0], [758.0, 249.0], [585.0, 249.0]],[[408.0, 252.0], [456.0, 252.0], [456.0, 270.0], [408.0, 270.0]],[[585.0, 250.0], [711.0, 247.0], [711.0, 268.0], [585.0, 271.0]],[[319.0, 276.0], [521.0, 276.0], [521.0, 290.0], [319.0, 290.0]],[[317.0, 291.0], [388.0, 291.0], [388.0, 312.0], [317.0, 312.0]],[[318.0, 315.0], [494.0, 316.0], [494.0, 330.0], [318.0, 329.0]],[[319.0, 331.0], [493.0, 331.0], [493.0, 351.0], [319.0, 351.0]],[[320.0, 359.0], [511.0, 359.0], [511.0, 373.0], [320.0, 373.0]],[[585.0, 357.0], [744.0, 357.0], [744.0, 371.0], [585.0, 371.0]],[[319.0, 377.0], [531.0, 377.0], [531.0, 401.0], [319.0, 401.0]],[[585.0, 371.0], [711.0, 371.0], [711.0, 392.0], [585.0, 392.0]],[[390.0, 422.0], [496.0, 422.0], [496.0, 446.0], [390.0, 446.0]],[[588.0, 422.0], [725.0, 425.0], [725.0, 452.0], [588.0, 450.0]]]", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)[0].split("Response")[-1]

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


':\n\nThe gender of the person in the image is not specified in the provided information.<|im_end|>'

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "what is the gender(use your creativity if not found concrete information)",
        "['REPUBLIOUE FRANCAISE',  'FR',  'CARTENATIONALEDIDENTITE/IDENTITYCARD',  'NOM/Sumame',  'MARTIN',  'Prenoms/Given.names',  'Maelys-Gaelle,Marie',  'SEXE/Sex',  'NATIONALITE/Nationalit',  'DATE DE NAlSS.YDate of birth',  'FRA',  '13071990',  'LIEUDE NAISSANCE/Place of birth',  'PARIS',  'NOM D'USAGE/ Altenate name',  'NOM D USAGE',  'NDU DOCUMENTLDocument No',  'DATE DEXPIR./Expiry date',  'X4RTBPFW4',  '11022030',  'Gnature',  '384213'],[[[184.0, 44.0], [608.0, 46.0], [608.0, 70.0], [184.0, 68.0]],[[668.0, 53.0], [717.0, 53.0], [717.0, 80.0], [668.0, 80.0]],[[185.0, 80.0], [609.0, 80.0], [609.0, 93.0], [185.0, 93.0]],[[313.0, 102.0], [408.0, 105.0], [408.0, 120.0], [313.0, 118.0]],[[317.0, 120.0], [431.0, 120.0], [431.0, 144.0], [317.0, 144.0]],[[319.0, 173.0], [447.0, 173.0], [447.0, 187.0], [319.0, 187.0]],[[317.0, 187.0], [558.0, 186.0], [558.0, 206.0], [317.0, 207.0]],[[317.0, 234.0], [388.0, 234.0], [388.0, 251.0], [317.0, 251.0]],[[410.0, 236.0], [553.0, 236.0], [553.0, 250.0], [410.0, 250.0]],[[585.0, 235.0], [758.0, 235.0], [758.0, 249.0], [585.0, 249.0]],[[408.0, 252.0], [456.0, 252.0], [456.0, 270.0], [408.0, 270.0]],[[585.0, 250.0], [711.0, 247.0], [711.0, 268.0], [585.0, 271.0]],[[319.0, 276.0], [521.0, 276.0], [521.0, 290.0], [319.0, 290.0]],[[317.0, 291.0], [388.0, 291.0], [388.0, 312.0], [317.0, 312.0]],[[318.0, 315.0], [494.0, 316.0], [494.0, 330.0], [318.0, 329.0]],[[319.0, 331.0], [493.0, 331.0], [493.0, 351.0], [319.0, 351.0]],[[320.0, 359.0], [511.0, 359.0], [511.0, 373.0], [320.0, 373.0]],[[585.0, 357.0], [744.0, 357.0], [744.0, 371.0], [585.0, 371.0]],[[319.0, 377.0], [531.0, 377.0], [531.0, 401.0], [319.0, 401.0]],[[585.0, 371.0], [711.0, 371.0], [711.0, 392.0], [585.0, 392.0]],[[390.0, 422.0], [496.0, 422.0], [496.0, 446.0], [390.0, 446.0]],[[588.0, 422.0], [725.0, 425.0], [725.0, 452.0], [588.0, 450.0]]]", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)[0].split("Response")[-1]

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


':\n\nThe gender of the person in the image is not explicitly mentioned. However, the name "Maelys-Gaelle,Marie" suggests that the person might be female.<|im_end|>'

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "what is the gender(use your creativity if not found concrete information)",
        "['REPUBLIOUE FRANCAISE',  'FR',  'CARTENATIONALEDIDENTITE/IDENTITYCARD',  'NOM/Sumame',  'MARTIN',  'Prenoms/Given.names',  'Maelys-Gaelle,Marie',  'SEXE/Sex',  'NATIONALITE/Nationalit',  'DATE DE NAlSS.YDate of birth',  'FRA',  '13071990',  'LIEUDE NAISSANCE/Place of birth',  'PARIS',  'NOM D'USAGE/ Altenate name',  'NOM D USAGE',  'NDU DOCUMENTLDocument No',  'DATE DEXPIR./Expiry date',  'X4RTBPFW4',  '11022030',  'Gnature',  '384213'],[[[184.0, 44.0], [608.0, 46.0], [608.0, 70.0], [184.0, 68.0]],[[668.0, 53.0], [717.0, 53.0], [717.0, 80.0], [668.0, 80.0]],[[185.0, 80.0], [609.0, 80.0], [609.0, 93.0], [185.0, 93.0]],[[313.0, 102.0], [408.0, 105.0], [408.0, 120.0], [313.0, 118.0]],[[317.0, 120.0], [431.0, 120.0], [431.0, 144.0], [317.0, 144.0]],[[319.0, 173.0], [447.0, 173.0], [447.0, 187.0], [319.0, 187.0]],[[317.0, 187.0], [558.0, 186.0], [558.0, 206.0], [317.0, 207.0]],[[317.0, 234.0], [388.0, 234.0], [388.0, 251.0], [317.0, 251.0]],[[410.0, 236.0], [553.0, 236.0], [553.0, 250.0], [410.0, 250.0]],[[585.0, 235.0], [758.0, 235.0], [758.0, 249.0], [585.0, 249.0]],[[408.0, 252.0], [456.0, 252.0], [456.0, 270.0], [408.0, 270.0]],[[585.0, 250.0], [711.0, 247.0], [711.0, 268.0], [585.0, 271.0]],[[319.0, 276.0], [521.0, 276.0], [521.0, 290.0], [319.0, 290.0]],[[317.0, 291.0], [388.0, 291.0], [388.0, 312.0], [317.0, 312.0]],[[318.0, 315.0], [494.0, 316.0], [494.0, 330.0], [318.0, 329.0]],[[319.0, 331.0], [493.0, 331.0], [493.0, 351.0], [319.0, 351.0]],[[320.0, 359.0], [511.0, 359.0], [511.0, 373.0], [320.0, 373.0]],[[585.0, 357.0], [744.0, 357.0], [744.0, 371.0], [585.0, 371.0]],[[319.0, 377.0], [531.0, 377.0], [531.0, 401.0], [319.0, 401.0]],[[585.0, 371.0], [711.0, 371.0], [711.0, 392.0], [585.0, 392.0]],[[390.0, 422.0], [496.0, 422.0], [496.0, 446.0], [390.0, 446.0]],[[588.0, 422.0], [725.0, 425.0], [725.0, 452.0], [588.0, 450.0]]]", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")



from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


<s> Below is an instruction that describes a task of layout question answering , paired with an input that provides further context that contain  a list of extracted blocks from that image then followed by a list of theirs bounding boxes represented by 4 point's cordinates of them in that image ,. Write a response that appropriately completes the request .

### Instruction:
what is the gender(use your creativity if not found concrete information)

### Input:
['REPUBLIOUE FRANCAISE',  'FR',  'CARTENATIONALEDIDENTITE/IDENTITYCARD',  'NOM/Sumame',  'MARTIN',  'Prenoms/Given.names',  'Maelys-Gaelle,Marie',  'SEXE/Sex',  'NATIONALITE/Nationalit',  'DATE DE NAlSS.YDate of birth',  'FRA',  '13071990',  'LIEUDE NAISSANCE/Place of birth',  'PARIS',  'NOM D'USAGE/ Altenate name',  'NOM D USAGE',  'NDU DOCUMENTLDocument No',  'DATE DEXPIR./Expiry date',  'X4RTBPFW4',  '11022030',  'Gnature',  '384213'],[[[184.0, 44.0], [608.0, 46.0], [608.0, 70.0], [184.0, 68.0]],[[668.0, 53.0], [717.0, 53.0], [7

Up to 6.1 vram in inference

# Expose the Model API using FLASK and ngrok free host  server

HOST PUBLIC API

use flask

In [ ]:
!pip install pyngrok

authentification

In [ ]:
!ngrok config add-authtoken 2ew195GVVcciKtWUZw4GrDwbpMZ_81fEV6QhZFrHn3cb1afHC


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install Flask


In [ ]:
!pip install flask
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip


--2024-04-22 22:32:44--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 52.202.168.65, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  19.5MB/s    in 0.7s    

2024-04-22 22:32:45 (19.5 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
from flask import Flask, request, jsonify, abort
from pyngrok import ngrok
import json

app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World!'

@app.route('/post', methods=['POST'])
def post_example():
    data = request.json
    texts = data.get('texts')
    boxes = data.get('boxes')
    nested_boxes = json.dumps(boxes)
    nested_texts = json.dumps(texts)
    instruction = data.get('instruction')

    # Assuming model and tokenizer are already defined elsewhere in your code
    inputs = tokenizer([
        alpaca_prompt.format(
            " ".join(instruction)+ "(use your creativity if not found concrete information)",
            nested_texts+" "+nested_boxes, # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors="pt").to("cuda")

    try:
        outputs = model.generate(**inputs, max_new_tokens=1000, use_cache=True)
        answer = tokenizer.batch_decode(outputs)[0].split("Response")[-1]
        if not answer:
            raise ValueError("Empty answer")
    except Exception as e:
        # If there's an error, raise a 500 HTTP error
        abort(500, description=str(e))

    return jsonify({"answer": answer})

# Set the address and port of your Flask application
address = 'localhost'
port = 8000

# Open an HTTP tunnel on the default ngrok port 80
ngrok_tunnel = ngrok.connect(addr=f'{address}:{port}')

# Get the public URL of the tunnel
public_url = ngrok_tunnel.public_url
print("Public URL:", public_url)

# Start the Flask application
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=port)


Public URL: https://5aa1-34-125-74-41.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://172.28.0.12:8000
INFO:werkzeug:Press CTRL+C to quit
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [22/Apr/2024 23:39:02] "POST /post HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [22/Apr/2024 23:39:38] "POST /post HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [22/Apr/2024 23:40:07] "POST /post HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [22/Apr/2024 23:40:23] "POST /post HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [22/Apr/2024 23: